# Example with ACCESS-CM2 data

In [1]:
import cmor
import xarray as xr
import numpy as np
import pandas as pd
import glob
import os

# Define the file pattern (e.g., all .nc files in a specific directory)
file_pattern = "/home/romain/PROJECTS/ACCESS-MOPPeR/Test_data/di787/history/atm/netCDF/di787a.pm*.nc"  # Adjust to your file pattern

# Use glob to match files based on the pattern
file_paths = glob.glob(file_pattern)

# Open the matching files with xarray
ds = xr.open_mfdataset(file_paths, combine='by_coords')

In [2]:
# Example mapping of variable names
mapping = {
    "hur": "fld_s30i296"
}

In [3]:
# Extract required variables and coordinates
hfls = ds[mapping["hur"]].values
lat = ds["lat"].values
lat_bnds = ds["lat_bnds"].isel(time=0).values
lon = ds["lon"].values
lon_bnds = ds["lon_bnds"].isel(time=0).values

KeyError: "No variable named 'fld_s30i296'. Variables on the dataset include ['fld_s00i023', 'latitude_longitude', 'time_bnds', 'lat_bnds', 'lon_bnds', ..., 'theta_level_height', 'model_theta_level_number', 'sigma_theta', 'surface_altitude', 'depth']"

In [ ]:
# Convert time to numeric values using xarray's datetime accessor
reference_time = "1850-01-01 00:00:00"  # Desired reference time
time_units = f"days since {reference_time}"
reference_date = pd.Timestamp(reference_time)

# Convert time to numeric values (days since reference_time)
time_numeric = (ds["time"] - np.datetime64(reference_date)).dt.days.values

# Handle time bounds
time_bnds = (ds["time_bnds"] - np.datetime64(reference_date)).dt.days.values

# CMOR setup
ipth = opth = "Test"
cmor.setup(inpath=ipth,
           set_verbosity=cmor.CMOR_NORMAL,
           netcdf_file_action=cmor.CMOR_REPLACE)
cmor.dataset_json("test.json")
cmor.load_table("CMIP6_Amon.json")

# Define CMOR axes
cmorLat = cmor.axis("latitude",
                    coord_vals=lat,
                    cell_bounds=lat_bnds,
                    units="degrees_north")
cmorLon = cmor.axis("longitude",
                    coord_vals=lon,
                    cell_bounds=lon_bnds,
                    units="degrees_east")
cmorTime = cmor.axis("time",
                     coord_vals=time_numeric,
                     cell_bounds=time_bnds,
                     units=time_units)

# Define CMOR variable
axes = [cmorTime, cmorLat, cmorLon]
cmorHfls = cmor.variable("hfls", "W/m2", axes, positive="up")

# Write data to CMOR
status = cmor.write(cmorHfls, hfls, ntimes_passed=len(time_numeric))

# Finalize and save the file
filename = cmor.close(cmorHfls, file_name=True)
print("Stored in:", filename)

cmor.close()

Stored in: MOPPeR_outputs/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r3i1p1f1/Amon/hfls/gn/v20250219/hfls_Amon_ACCESS-CM2_historical_r3i1p1f1_gn_201101-201212.nc



! ------
! All files were closed successfully. 
! ------
! 


0